## Exploring the data

So let's learn how we set up the data. 

In [1]:
#default_exp model

In [ ]:
#hide
import os, sys, warnings

In [ ]:
#hide
root = "D:/data_sets/24_garden"
#os.chdir(root)

In [ ]:
#hide
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
#export
from fastai2.vision.all import *
from garden2.utils import *
from garden2.data import *

In [ ]:
splitter = IndexSplitter(range(12753, len(data)))

block = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                  get_items=get_items,
                  get_x=get_x,
                  get_y=get_y,
                  splitter=splitter,
                  item_tfms=[Resize(224)],
                  batch_tfms=[*aug_transforms()])

In [ ]:
dls = block.dataloaders('', bs=64,num_workers=0)

In [ ]:
dls.show_batch()

In [ ]:
lr = 1e-2
def opt_func(ps, lr=defaults.lr): ranger(ps,lr, mom=0.98, wd=0.02, eps=1e-06, 
                  sqr_mom=0.99, beta=0.0, decouple_wd=True)
net1 = xresnet50(pretrained=False, sa=True, act_cls=Mish)
learn1 = Learner(dls, net1, opt_func=ranger, metrics=accuracy)
learn1.fit_one_cycle(3, 1e-2)
learn1.fit_one_cycle(4, 2e-2)

In [ ]:
learn1.fit_one_cycle(1, 1e-5)

In [ ]:
preds1, targs1 = learn1.get_preds()

In [ ]:
net2 = xresnet50(pretrained=False, sa=True, act_cls=Mish)
learn2 = Learner(dls, net2, opt_func=ranger, metrics=accuracy)
learn2.fit_flat_cos(5, 1e-2)

In [ ]:
preds2, _ = learn2.get_preds()

In [ ]:
#Combining predictions

In [ ]:
preds = torch.stack((preds1, preds2))
preds.std()

In [ ]:
preds.shape

In [ ]:
import numpy as np
np.corrcoef(preds.view(preds.shape[0], -1))

In [ ]:
accuracy(preds.mean(0), targs)

In [ ]:
#hide
from nbdev.export import *
from nbdev.imports import *
os.chdir(root+'/nbs_dev')
notebook2script()